In [1]:
import pandas as pd
import numpy as np

In [2]:
DiabetesTakingMed = pd.read_csv('DiabetesTakingMed.csv', index_col=0)

DiabetesTrain3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==1].drop('IsTrain', axis=1)
DiabetesTrain3.index = list(range(len(DiabetesTrain3)))

DiabetesTest3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==0].drop('IsTrain', axis=1)
DiabetesTest3.index = list(range(len(DiabetesTest3)))

#Start with a train to predict the placement of the middle group:

DiabetesTrain3 = DiabetesTrain3[DiabetesTrain3['readmitted']!=1]
trainX01 = DiabetesTrain3.drop('readmitted', axis=1)
trainY01 = DiabetesTrain3['readmitted'].replace([2], [1])

testX01 = DiabetesTest3.drop('readmitted', axis=1)
testY01 = DiabetesTest3['readmitted'].replace([2], [1])

In [3]:
from sklearn.linear_model import LinearRegression as lm

lm = lm()

lm.fit(trainX01, trainY01)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [4]:
DiabetesTakingMed = pd.read_csv('DiabetesTakingMed.csv', index_col=0)

DiabetesTrain3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==1].drop('IsTrain', axis=1)
DiabetesTrain3.index = list(range(len(DiabetesTrain3)))

middledf = DiabetesTrain3[DiabetesTrain3['readmitted']==1]
middledfX = middledf.drop('readmitted', axis=1)
middledfY = middledf['readmitted']

predictarray = lm.predict(middledfX)

In [5]:
len(predictarray[predictarray>0.5])

1171

In [6]:
len(predictarray)

28436

In [7]:
middledf100 = middledf.loc[predictarray<1]
middledf75 = middledf.loc[predictarray<0.75]
middledf50 = middledf.loc[predictarray<0.50]
middledf25 = middledf.loc[predictarray<0.25]

In [8]:
trainX01.shape

(51648, 83)

In [9]:
DiabetesTakingMed = pd.read_csv('DiabetesTakingMed.csv', index_col=0)

DiabetesTrain3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==1].drop('IsTrain', axis=1)
DiabetesTrain3.index = list(range(len(DiabetesTrain3)))
DiabetesTrain3 = DiabetesTrain3[DiabetesTrain3['readmitted']!=1]

train25 = pd.concat([DiabetesTrain3, middledf25], axis=0)
train25.index = list(range(len(train25)))

In [10]:
train50 = pd.concat([DiabetesTrain3, middledf50], axis=0)
train50.index = list(range(len(train50)))

train75 = pd.concat([DiabetesTrain3, middledf75], axis=0)
train75.index = list(range(len(train75)))

train100 = pd.concat([DiabetesTrain3, middledf100], axis=0)
train100.index = list(range(len(train100)))

In [11]:
#First, try logistic regression with all observations:

DiabetesTakingMed = pd.read_csv('DiabetesTakingMed.csv', index_col=0)

DiabetesTrain3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==1].drop('IsTrain', axis=1)
DiabetesTrain3.index = list(range(len(DiabetesTrain3)))

trainX02 = DiabetesTrain3.drop('readmitted', axis=1)
trainY02 = DiabetesTrain3['readmitted'].replace([2, 1], [1, 0])

testX02 = DiabetesTest3.drop('readmitted', axis=1)
testY02 = DiabetesTest3['readmitted'].replace([2, 1], [1, 0])

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.1)

lgr.fit(trainX02, trainY02)
predictprobs02 = lgr.predict_proba(trainX02)
predictvalues02 = lgr.predict(trainX02)
actual02 = trainY02.values

predicttest02 = lgr.predict(testX02)
predicttestprobs02 = lgr.predict_proba(testX02)

In [12]:
from sklearn.metrics import roc_auc_score as AUC

AUC(testY02, predicttestprobs02[:,1])

0.6633660039876829

In [15]:
#Now, let's try shaving off a few of the observations at a time: (Any >30 with linear P prediction over 1.0)

trainX03 = train100.drop('readmitted', axis=1)
trainY03 = train100['readmitted'].replace([2, 1], [1, 0])

testX03 = DiabetesTest3.drop('readmitted', axis=1)
testY03 = DiabetesTest3['readmitted'].replace([2, 1], [1, 0])

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.1)

lgr.fit(trainX03, trainY03)
predictprobs03 = lgr.predict_proba(trainX03)
predictvalues03 = lgr.predict(trainX03)
actual03 = trainY03.values

predicttest03 = lgr.predict(testX03)
predicttestprobs03 = lgr.predict_proba(testX03)

In [16]:
from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predicttestprobs03[:,1])

0.6640227247295477

In [17]:
#Now, let's try shaving off a few of the observations at a time: (Any >30 with linear P prediction over 0.75)

trainX04 = train75.drop('readmitted', axis=1)
trainY04 = train75['readmitted'].replace([2, 1], [1, 0])

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.1)

lgr.fit(trainX04, trainY04)
predictprobs04 = lgr.predict_proba(trainX04)
predictvalues04 = lgr.predict(trainX04)
actual04 = trainY04.values

predicttest04 = lgr.predict(testX03)
predicttestprobs04 = lgr.predict_proba(testX03)

In [18]:
from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predicttestprobs04[:,1])

0.6641325666417452

In [19]:
#Now, let's try shaving off a few of the observations at a time: (Any >30 with linear P prediction over 0.50)

trainX05 = train50.drop('readmitted', axis=1)
trainY05 = train50['readmitted'].replace([2, 1], [1, 0])

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.1)

lgr.fit(trainX05, trainY05)
predictprobs05 = lgr.predict_proba(trainX05)
predictvalues05 = lgr.predict(trainX05)
actual05 = trainY05.values

predicttest05 = lgr.predict(testX03)
predicttestprobs05 = lgr.predict_proba(testX03)

In [20]:
from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predicttestprobs05[:,1])

0.6632222672912926

In [21]:
#We see that 0.75 was better then 1.0 and much better than 0.5. Let's try 0.7, 0.8, 0.9:
middledf90 = middledf.loc[predictarray<0.9]
middledf80 = middledf.loc[predictarray<0.8]
middledf70 = middledf.loc[predictarray<0.7]

In [22]:
DiabetesTakingMed = pd.read_csv('DiabetesTakingMed.csv', index_col=0)

DiabetesTrain3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==1].drop('IsTrain', axis=1)
DiabetesTrain3.index = list(range(len(DiabetesTrain3)))
DiabetesTrain3 = DiabetesTrain3[DiabetesTrain3['readmitted']!=1]

train90 = pd.concat([DiabetesTrain3, middledf90], axis=0)
train90.index = list(range(len(train90)))

train80 = pd.concat([DiabetesTrain3, middledf80], axis=0)
train80.index = list(range(len(train80)))

train70 = pd.concat([DiabetesTrain3, middledf70], axis=0)
train70.index = list(range(len(train70)))

In [23]:
#Now, let's try shaving off a few of the observations at a time: (Any >30 with linear P prediction over 0.90)

trainX06 = train90.drop('readmitted', axis=1)
trainY06 = train90['readmitted'].replace([2, 1], [1, 0])

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.1)

lgr.fit(trainX06, trainY06)

predicttestprobs06 = lgr.predict_proba(testX03)

In [24]:
from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predicttestprobs06[:,1])

0.6639589519534368

In [25]:
#Now, let's try shaving off a few of the observations at a time: (Any >30 with linear P prediction over 0.80)

trainX07 = train80.drop('readmitted', axis=1)
trainY07 = train80['readmitted'].replace([2, 1], [1, 0])

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.1)

lgr.fit(trainX07, trainY07)

predicttestprobs07 = lgr.predict_proba(testX03)

In [26]:
from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predicttestprobs07[:,1])

0.6640319732697846

In [27]:
#Now, let's try shaving off a few of the observations at a time: (Any >30 with linear P prediction over 0.80)

trainX08 = train70.drop('readmitted', axis=1)
trainY08 = train70['readmitted'].replace([2, 1], [1, 0])

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.1)

lgr.fit(trainX08, trainY08)

predicttestprobs08 = lgr.predict_proba(testX03)

In [28]:
from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predicttestprobs08[:,1])

0.6640376017379718

In [29]:
#Based on this, the best score we have is 0.75. If we remove the returned>30 observations with linear-predcted likelihood of returning >0.75

In [30]:
#Call Train75, and Train75 our new training DF

trainX = train75.drop('readmitted', axis=1)
trainY = train75['readmitted'].replace([2, 1], [1, 0])

In [31]:
#We can write a function to find a candidate feature list using AIC engineering:

def FindLowestAICNonLogBackward(df, dependent):
    '''Input: DF to AIC-modify and the dependent variable. WILL RETURN: A tuple: [0] is the modified DF (with dependent)
    and tuple[1] will give you the summary DF'''
    df2 = df.copy()
    df2X = df2.drop(dependent, axis=1)
    df2Y = df2[dependent]
    FeatureList = list(df2X.columns)
    X2 = sm.add_constant(df2X)
    est = sm.OLS(df2Y, X2)
    CurrentAIC = est.fit().aic
    CanBeBetter = True
    ModList = []
    AddedSubtracted = []
    AIC = []
    TriesSinceReset = 0
    
    tempColumnList = list(df2X.columns)
    tempDF2X = df2X[tempColumnList]
    
    while CanBeBetter == True:
        Choice = np.random.choice(list(df2X.columns))
        
        HeadsTails = np.random.randint(2)
        if Choice in tempColumnList:
            HeadsTails = 0
        if Choice not in tempColumnList:
            HeadsTails = 1
        
        if HeadsTails == 1:
            tempColumnList.append(Choice)
            tempDF2X[Choice] = df2X[Choice]
        
        if HeadsTails == 0:

            tempColumnList.remove(Choice)
            tempDF2X = tempDF2X[tempColumnList]
            
        est = sm.OLS(df2Y, sm.add_constant(tempDF2X))
        NewAIC = est.fit().aic
        
        if NewAIC < CurrentAIC:
            TriesSinceReset = 0
            CurrentAIC = NewAIC
            
            if HeadsTails == 1:
                print(Choice + " added: New AIC = " + str(CurrentAIC))
                ModList.append(Choice)
                AddedSubtracted.append('Added')
                AIC.append(CurrentAIC)
                
            if HeadsTails == 0:
                print(Choice + " removed: New AIC = " + str(CurrentAIC))
                ModList.append(Choice)
                AddedSubtracted.append('Subtracted')
                AIC.append(CurrentAIC)
            continue
            
        else:
            TriesSinceReset += 1
            
            if HeadsTails == 1:
                tempColumnList.remove(Choice)
                tempDF2X = tempDF2X[tempColumnList]
                
            if HeadsTails == 0:
                tempColumnList.append(Choice)
                tempDF2X[Choice] = df2X[Choice]
                
            if TriesSinceReset > 100:
                CanBeBetter = False
                
    SummaryDF = pd.DataFrame({'Feature': ModList, 'AddOrSubtract': AddedSubtracted, 'AIC': AIC})
    NewDF = pd.concat([tempDF2X, df2[[dependent]]], axis=1)
    
    return NewDF, SummaryDF  

In [32]:
import statsmodels.api as sm
trainXAIC = FindLowestAICNonLogBackward(train75, 'readmitted')[0]

diag_mentaldis removed: New AIC = 159162.03706622205
change_Ch removed: New AIC = 159160.51964032956
med_repaglinide removed: New AIC = 159160.0911285198
med_rosiglitazone removed: New AIC = 159159.8973074622
med_glyburide removed: New AIC = 159159.19290812098
diag_other removed: New AIC = 159157.51199187117
diag_infection removed: New AIC = 159157.4832474055
med_chlorpropamide removed: New AIC = 159155.4835727667
med_glipizide.metformin removed: New AIC = 159153.625796249
diabfeat_coma removed: New AIC = 159151.66326763172
diag_injury removed: New AIC = 159150.08370077435
discharge_disposition_unknown removed: New AIC = 159148.45328974957
med_nateglinide removed: New AIC = 159146.4608589305
diag_urogenital removed: New AIC = 159144.52989303845
race_AfricanAmerican removed: New AIC = 159144.34920693107
admission_type_id_3 removed: New AIC = 159142.45471779804
med_glyburide.metformin removed: New AIC = 159140.7764499014
diag_nan removed: New AIC = 159138.87077050714
med_tolazamide remov

In [33]:
train10 = train75[list(trainXAIC.columns)]
trainX10 = train10.drop('readmitted', axis=1)
trainY10 = train10['readmitted'].replace([2, 1], [1, 0])

DiabetesTakingMed = pd.read_csv('DiabetesTakingMed.csv', index_col=0)

DiabetesTrain3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==1].drop('IsTrain', axis=1)
DiabetesTrain3.index = list(range(len(DiabetesTrain3)))

DiabetesTest3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==0].drop('IsTrain', axis=1)
DiabetesTest3.index = list(range(len(DiabetesTest3)))

testX10 = DiabetesTest3[list(trainXAIC.columns)].drop('readmitted', axis=1)

from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.1)

lgr.fit(trainX10, trainY10)

predicttestprobs10 = lgr.predict_proba(testX10)

In [34]:
#This made everything worse. Wonderful

from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predicttestprobs10[:,1])

0.6624476214626535

In [35]:
from xgboost.sklearn import XGBClassifier as xgb

xgb = xgb()
xgb.set_params(n_estimators=500, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, max_delta_step=5)

xgb.fit(trainX, trainY)

predictXB = xgb.predict(testX03)
predictprobsXB = xgb.predict_proba(testX03)

C:\Users\Dave\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predictprobsXB[:,1])

0.6768904939858578

In [37]:
# AUC of 0.66413 for logistic regression
# AUC of 0.67689 for parameter-optimized XGBoost

In [38]:
from sklearn.ensemble import RandomForestClassifier as rfc

rfc = rfc()
rfc.set_params(n_estimators=1000, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', max_depth=60)

rfc.fit(trainX, trainY)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=60, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [39]:
predictRF = rfc.predict(testX03)
predictprobsRF = rfc.predict_proba(testX03)

In [40]:
from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predictprobsRF[:,1])

0.6617318265998382

In [41]:
# AUC of 0.66413 for logistic regression
# AUC of 0.67689 for parameter-optimized XGBoost
# AUC of 0.66173 for random forest

In [42]:
from sklearn.neighbors import KNeighborsClassifier as knn

knn = knn()
knn.set_params(n_neighbors=10)

knn.fit(trainX, trainY)

predictKN = knn.predict(testX03)
predictprobsKN = knn.predict_proba(testX03)

In [43]:
#KNN is atrocious
from sklearn.metrics import roc_auc_score as AUC

AUC(testY03, predictprobsKN[:,1])

0.5551912961367955

In [44]:
len(predictarray[predictarray>1])

60

In [45]:
len(predictarray[predictarray<0])

8

In [46]:
#We have a predictarray essentially bounded by 0 and 1, with very few exceptions

In [47]:
#We also have an average of approximately 20%
np.mean(predictarray)

0.21584492882744952

In [48]:
predictarrayF = predictarray.copy()

predictarrayF[predictarrayF<0] = 0
predictarrayF[predictarrayF>1] = 1

In [54]:
#What if we shuffled our data to reflect differences in probability of these events occurring?

DiabetesAltered = pd.read_csv('DiabetesTakingMed.csv', index_col=0)
DiabetesAlteredTrain = DiabetesAltered[DiabetesAltered['IsTrain']==1]

In [58]:
DiabetesAlteredTrain['readmittedprob'] = DiabetesAlteredTrain['readmitted'].copy()
DiabetesAlteredTrain.loc[DiabetesAlteredTrain['readmittedprob']==1,'readmittedprob'] = predictarrayF

C:\Users\Dave\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Dave\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [66]:
#Set 'readmitted' column to the correct values (2 to 1, 1 to 0):
DiabetesAlteredTrain['readmitted'] = DiabetesAlteredTrain['readmitted'].replace([2, 1], [1, 0])

C:\Users\Dave\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [61]:
'''Right now, we have all returns <30 days coded as 2; all returns >30 days coded between 0 and 1 based on probability taken
from a linear regression trained on only the never-returning group and the <30 returning group.  We need to save a copy of
this DF because we will adjust the numbers based on how we want to tune them.'''

'Right now, we have all returns <30 days coded as 2; all returns >30 days coded between 0 and 1 based on probability taken\nfrom a linear regression trained on only the never-returning group and the <30 returning group.  We need to save a copy of\nthis DF because we will adjust the numbers based on how we want to tune them.'

In [76]:
#First, make a dataset where 1.0 values from the >30 group get equal weight to the <30 returning group:
DiabetesAlteredTrain100 = DiabetesAlteredTrain.copy()
DiabetesAlteredTrain100['readmittedprob'] = DiabetesAlteredTrain100['readmittedprob'].replace([2], [1])

In [77]:
#Make readmittedprob adjust to the total size of readmitted:
DiabetesAlteredTrain100['readmittedprob'] = (DiabetesAlteredTrain100['readmittedprob'] * 
                                             np.sum(DiabetesAlteredTrain100['readmitted'])/np.sum(DiabetesAlteredTrain100['readmittedprob']))

In [78]:
#This process has forced all positive values to shrink to ~60% probability, which may not be ideal. Let's find out:
DiabetesAlteredTrain100['readmittedprob'].describe()

count    80084.000000
mean         0.113456
std          0.189136
min          0.000000
25%          0.000000
50%          0.000000
75%          0.127133
max          0.597408
Name: readmittedprob, dtype: float64

In [82]:
problist = DiabetesAlteredTrain100['readmittedprob'].copy()

Diabetes100SetOf20 = DiabetesAlteredTrain100.copy()
Diabetes100SetOf20 = Diabetes100SetOf20.drop(['readmittedprob', 'IsTrain'], axis=1)
PredictedYesProbs = np.zeros(20030)

for i in range(20):
    newreadmitted = np.zeros(80084)
    for j in range(80084):
        newreadmitted[j] = np.random.choice([0,1], size=1, p=[1-problist[j], problist[j]])
    Diabetes100SetOf20['readmitted'] = pd.Series(newreadmitted)
    
    temptrainX = Diabetes100SetOf20.drop('readmitted', axis=1)
    temptrainY = Diabetes100SetOf20['readmitted']
    
    from sklearn.linear_model import LogisticRegression as lgr

    lgr = lgr()
    lgr.set_params(C=0.1)

    lgr.fit(temptrainX, temptrainY)

    PredictedYesProbs += lgr.predict_proba(testX03)[:,1]/20

In [83]:
#This is interesting. It worked, in the sense that we got an AUC, and it was only slightly lower than our normal LR AUC (0.66)
#Would toning it down give us a better score? (Assigning 1.0 in the midle category to a value less than the actual <30)
AUC(testY03, PredictedYesProbs)

0.6555740964405543

In [104]:
DiabetesAlteredTrain80 = DiabetesAlteredTrain.copy()
DiabetesAlteredTrain80['readmittedprob'] = 0.8*DiabetesAlteredTrain80['readmittedprob']
DiabetesAlteredTrain80['readmittedprob'] = DiabetesAlteredTrain80['readmittedprob'].replace([1.6], [1])

In [105]:
#Make readmittedprob adjust to the total size of readmitted:
DiabetesAlteredTrain80['readmittedprob'] = (DiabetesAlteredTrain80['readmittedprob'] * 
                                             np.sum(DiabetesAlteredTrain80['readmitted'])/np.sum(DiabetesAlteredTrain80['readmittedprob']))

DiabetesAlteredTrain80['readmittedprob'].describe()
#Now, the original <30's are only penalized down to 0.65

count    80084.000000
mean         0.113456
std          0.202962
min          0.000000
25%          0.000000
50%          0.000000
75%          0.110613
max          0.649722
Name: readmittedprob, dtype: float64

In [106]:
problist = DiabetesAlteredTrain80['readmittedprob'].copy()

Diabetes80SetOf20 = DiabetesAlteredTrain80.copy()
Diabetes80SetOf20 = Diabetes80SetOf20.drop(['readmittedprob', 'IsTrain'], axis=1)
PredictedYesProbs = np.zeros(20030)

for i in range(20):
    newreadmitted = np.zeros(80084)
    for j in range(80084):
        newreadmitted[j] = np.random.choice([0,1], size=1, p=[1-problist[j], problist[j]])
    Diabetes80SetOf20['readmitted'] = pd.Series(newreadmitted)
    
    temptrainX = Diabetes80SetOf20.drop('readmitted', axis=1)
    temptrainY = Diabetes80SetOf20['readmitted']
    
    from sklearn.linear_model import LogisticRegression as lgr

    lgr = lgr()
    lgr.set_params(C=0.1)

    lgr.fit(temptrainX, temptrainY)

    PredictedYesProbs += lgr.predict_proba(testX03)[:,1]/20

In [107]:
#Giving 80% of the value is still not ideal
AUC(testY03, PredictedYesProbs)

0.6571373724857514

In [108]:
DiabetesAlteredTrain60 = DiabetesAlteredTrain.copy()
DiabetesAlteredTrain60['readmittedprob'] = 0.6*DiabetesAlteredTrain60['readmittedprob']
DiabetesAlteredTrain60['readmittedprob'] = DiabetesAlteredTrain60['readmittedprob'].replace([1.2], [1])

#Make readmittedprob adjust to the total size of readmitted:
DiabetesAlteredTrain60['readmittedprob'] = (DiabetesAlteredTrain60['readmittedprob'] * 
                                             np.sum(DiabetesAlteredTrain60['readmitted'])/np.sum(DiabetesAlteredTrain60['readmittedprob']))

DiabetesAlteredTrain60['readmittedprob'].describe()
#At 60%, 1.0 penalized down to 71%

count    80084.000000
mean         0.113456
std          0.220967
min          0.000000
25%          0.000000
50%          0.000000
75%          0.090922
max          0.712079
Name: readmittedprob, dtype: float64

In [113]:
problist = DiabetesAlteredTrain60['readmittedprob'].copy()

Diabetes60SetOf20 = DiabetesAlteredTrain60.copy()
Diabetes60SetOf20 = Diabetes60SetOf20.drop(['readmittedprob', 'IsTrain'], axis=1)
PredictedYesProbs = np.zeros(20030)

for i in range(20):
    newreadmitted = np.zeros(80084)
    for j in range(80084):
        newreadmitted[j] = np.random.choice([0,1], size=1, p=[1-problist[j], problist[j]])
    Diabetes60SetOf20['readmitted'] = pd.Series(newreadmitted)
    
    temptrainX = Diabetes60SetOf20.drop('readmitted', axis=1)
    temptrainY = Diabetes60SetOf20['readmitted']
    
    from sklearn.linear_model import LogisticRegression as lgr

    lgr = lgr()
    lgr.set_params(C=0.1)

    lgr.fit(temptrainX, temptrainY)

    PredictedYesProbs += lgr.predict_proba(testX03)[:,1]/20
    
AUC(testY03, PredictedYesProbs)

0.6593965453949968

In [114]:
DiabetesAlteredTrain50 = DiabetesAlteredTrain.copy()
DiabetesAlteredTrain50['readmittedprob'] = 0.5*DiabetesAlteredTrain50['readmittedprob']
DiabetesAlteredTrain50['readmittedprob'] = DiabetesAlteredTrain50['readmittedprob'].replace([1.0], [1])

#Make readmittedprob adjust to the total size of readmitted:
DiabetesAlteredTrain50['readmittedprob'] = (DiabetesAlteredTrain50['readmittedprob'] * 
                                             np.sum(DiabetesAlteredTrain50['readmitted'])/np.sum(DiabetesAlteredTrain50['readmittedprob']))

DiabetesAlteredTrain50['readmittedprob'].describe()
#At 50%, 1.0 only penalized to 0.75

count    80084.000000
mean         0.113456
std          0.231943
min          0.000000
25%          0.000000
50%          0.000000
75%          0.079587
max          0.747971
Name: readmittedprob, dtype: float64

In [115]:
problist = DiabetesAlteredTrain50['readmittedprob'].copy()

Diabetes50SetOf20 = DiabetesAlteredTrain50.copy()
Diabetes50SetOf20 = Diabetes50SetOf20.drop(['readmittedprob', 'IsTrain'], axis=1)
PredictedYesProbs = np.zeros(20030)

for i in range(20):
    newreadmitted = np.zeros(80084)
    for j in range(80084):
        newreadmitted[j] = np.random.choice([0,1], size=1, p=[1-problist[j], problist[j]])
    Diabetes50SetOf20['readmitted'] = pd.Series(newreadmitted)
    
    temptrainX = Diabetes50SetOf20.drop('readmitted', axis=1)
    temptrainY = Diabetes50SetOf20['readmitted']
    
    from sklearn.linear_model import LogisticRegression as lgr

    lgr = lgr()
    lgr.set_params(C=0.1)

    lgr.fit(temptrainX, temptrainY)

    PredictedYesProbs += lgr.predict_proba(testX03)[:,1]/20
    
AUC(testY03, PredictedYesProbs)

0.6599108683712296

In [116]:
DiabetesAlteredTrain40 = DiabetesAlteredTrain.copy()
DiabetesAlteredTrain40['readmittedprob'] = 0.4*DiabetesAlteredTrain40['readmittedprob']
DiabetesAlteredTrain40['readmittedprob'] = DiabetesAlteredTrain40['readmittedprob'].replace([0.8], [1])

#Make readmittedprob adjust to the total size of readmitted:
DiabetesAlteredTrain40['readmittedprob'] = (DiabetesAlteredTrain40['readmittedprob'] * 
                                             np.sum(DiabetesAlteredTrain40['readmitted'])/np.sum(DiabetesAlteredTrain40['readmittedprob']))

DiabetesAlteredTrain40['readmittedprob'].describe()

count    80084.000000
mean         0.113456
std          0.244515
min          0.000000
25%          0.000000
50%          0.000000
75%          0.067050
max          0.787675
Name: readmittedprob, dtype: float64

In [118]:
problist = DiabetesAlteredTrain40['readmittedprob'].copy()

Diabetes40SetOf20 = DiabetesAlteredTrain40.copy()
Diabetes40SetOf20 = Diabetes40SetOf20.drop(['readmittedprob', 'IsTrain'], axis=1)
PredictedYesProbs = np.zeros(20030)

for i in range(20):
    newreadmitted = np.zeros(80084)
    for j in range(80084):
        newreadmitted[j] = np.random.choice([0,1], size=1, p=[1-problist[j], problist[j]])
    Diabetes40SetOf20['readmitted'] = pd.Series(newreadmitted)
    
    temptrainX = Diabetes40SetOf20.drop('readmitted', axis=1)
    temptrainY = Diabetes40SetOf20['readmitted']
    
    from sklearn.linear_model import LogisticRegression as lgr

    lgr = lgr()
    lgr.set_params(C=0.1)

    lgr.fit(temptrainX, temptrainY)

    PredictedYesProbs += lgr.predict_proba(testX03)[:,1]/20
    
AUC(testY03, PredictedYesProbs)

0.6609319766394272

In [123]:
DiabetesAlteredTrain30 = DiabetesAlteredTrain.copy()
DiabetesAlteredTrain30['readmittedprob'] = 0.3*DiabetesAlteredTrain30['readmittedprob']
DiabetesAlteredTrain30['readmittedprob'] = DiabetesAlteredTrain30['readmittedprob'].replace([0.6], [1])

#Make readmittedprob adjust to the total size of readmitted:
DiabetesAlteredTrain30['readmittedprob'] = (DiabetesAlteredTrain30['readmittedprob'] * 
                                             np.sum(DiabetesAlteredTrain30['readmitted'])/np.sum(DiabetesAlteredTrain30['readmittedprob']))

DiabetesAlteredTrain30['readmittedprob'].describe()

count    80084.000000
mean         0.113456
std          0.258951
min          0.000000
25%          0.000000
50%          0.000000
75%          0.053106
max          0.831829
Name: readmittedprob, dtype: float64

In [124]:
problist = DiabetesAlteredTrain30['readmittedprob'].copy()

Diabetes30SetOf20 = DiabetesAlteredTrain30.copy()
Diabetes30SetOf20 = Diabetes30SetOf20.drop(['readmittedprob', 'IsTrain'], axis=1)
PredictedYesProbs = np.zeros(20030)

for i in range(20):
    newreadmitted = np.zeros(80084)
    for j in range(80084):
        newreadmitted[j] = np.random.choice([0,1], size=1, p=[1-problist[j], problist[j]])
    Diabetes30SetOf20['readmitted'] = pd.Series(newreadmitted)
    
    temptrainX = Diabetes30SetOf20.drop('readmitted', axis=1)
    temptrainY = Diabetes30SetOf20['readmitted']
    
    from sklearn.linear_model import LogisticRegression as lgr

    lgr = lgr()
    lgr.set_params(C=0.1)

    lgr.fit(temptrainX, temptrainY)

    PredictedYesProbs += lgr.predict_proba(testX03)[:,1]/20
    
AUC(testY03, PredictedYesProbs)

0.6618649634277263

In [125]:
DiabetesAlteredTrain20 = DiabetesAlteredTrain.copy()
DiabetesAlteredTrain20['readmittedprob'] = 0.2*DiabetesAlteredTrain20['readmittedprob']
DiabetesAlteredTrain20['readmittedprob'] = DiabetesAlteredTrain20['readmittedprob'].replace([0.4], [1])

#Make readmittedprob adjust to the total size of readmitted:
DiabetesAlteredTrain20['readmittedprob'] = (DiabetesAlteredTrain20['readmittedprob'] * 
                                             np.sum(DiabetesAlteredTrain20['readmitted'])/np.sum(DiabetesAlteredTrain20['readmittedprob']))

DiabetesAlteredTrain20['readmittedprob'].describe()

count    80084.000000
mean         0.113456
std          0.275579
min          0.000000
25%          0.000000
50%          0.000000
75%          0.037507
max          0.881228
Name: readmittedprob, dtype: float64

In [127]:
problist = DiabetesAlteredTrain20['readmittedprob'].copy()

Diabetes20SetOf20 = DiabetesAlteredTrain20.copy()
Diabetes20SetOf20 = Diabetes20SetOf20.drop(['readmittedprob', 'IsTrain'], axis=1)
PredictedYesProbs = np.zeros(20030)

for i in range(20):
    newreadmitted = np.zeros(80084)
    for j in range(80084):
        newreadmitted[j] = np.random.choice([0,1], size=1, p=[1-problist[j], problist[j]])
    Diabetes20SetOf20['readmitted'] = pd.Series(newreadmitted)
    
    temptrainX = Diabetes20SetOf20.drop('readmitted', axis=1)
    temptrainY = Diabetes20SetOf20['readmitted']
    
    from sklearn.linear_model import LogisticRegression as lgr

    lgr = lgr()
    lgr.set_params(C=0.1)

    lgr.fit(temptrainX, temptrainY)

    PredictedYesProbs += lgr.predict_proba(testX03)[:,1]/20
    
AUC(testY03, PredictedYesProbs)

0.6627061590740491

In [129]:
#Let's try a 40% XGB and see if that is any better than a base 0% xgb:

from xgboost.sklearn import XGBClassifier as xgb2

problist = DiabetesAlteredTrain40['readmittedprob'].copy()

Diabetes40SetOf20 = DiabetesAlteredTrain40.copy()
Diabetes40SetOf20 = Diabetes40SetOf20.drop(['readmittedprob', 'IsTrain'], axis=1)
PredictedYesProbs = np.zeros(20030)

for i in range(20):
    newreadmitted = np.zeros(80084)
    for j in range(80084):
        newreadmitted[j] = np.random.choice([0,1], size=1, p=[1-problist[j], problist[j]])
    Diabetes40SetOf20['readmitted'] = pd.Series(newreadmitted)
    
    temptrainX = Diabetes40SetOf20.drop('readmitted', axis=1)
    temptrainY = Diabetes40SetOf20['readmitted']
    
    xgb = xgb2()
    xgb.set_params(n_estimators=500, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, max_delta_step=5)

    xgb.fit(temptrainX, temptrainY)

    PredictedYesProbs += xgb.predict_proba(testX03)[:,1]/20
    
AUC(testY03, PredictedYesProbs)

0.6745317195052136

In [130]:
#Let's try a 20% XGB and see if that is any better than a base 0% xgb:

from xgboost.sklearn import XGBClassifier as xgb2

problist = DiabetesAlteredTrain20['readmittedprob'].copy()

Diabetes20SetOf20 = DiabetesAlteredTrain20.copy()
Diabetes20SetOf20 = Diabetes20SetOf20.drop(['readmittedprob', 'IsTrain'], axis=1)
PredictedYesProbs = np.zeros(20030)

for i in range(20):
    newreadmitted = np.zeros(80084)
    for j in range(80084):
        newreadmitted[j] = np.random.choice([0,1], size=1, p=[1-problist[j], problist[j]])
    Diabetes40SetOf20['readmitted'] = pd.Series(newreadmitted)
    
    temptrainX = Diabetes20SetOf20.drop('readmitted', axis=1)
    temptrainY = Diabetes20SetOf20['readmitted']
    
    xgb = xgb2()
    xgb.set_params(n_estimators=500, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, max_delta_step=5)

    xgb.fit(temptrainX, temptrainY)

    PredictedYesProbs += xgb.predict_proba(testX03)[:,1]/20
    
AUC(testY03, PredictedYesProbs)

0.6753665800254492

In [158]:
devontrain = pd.read_csv('db_train.csv')
devontest = pd.read_csv('db_test.csv')

devontrain['IsTrain'] = 1
devontest['IsTrain'] = 0

devonDF = pd.concat([devontrain, devontest], axis=0)

In [160]:
devonDF = devonDF.astype('str')

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for item in ['race', 'gender', 'age', 'A1Cresult', 'metformin', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 
             'insulin', 'change', 'diabetesMed', 'diag_1_cat', 'diag_Circulatory', 'diag_Diabetes', 'diag_Digestive', 
             'diag_Genitourinary', 'diag_Injury', 'diag_Musculoskeletal', 'diag_Neoplasms', 'diag_Other', 
             'diag_Respiratory', 'admission_type_id_cat', 'discharge_disposition_id_cat', 'admission_source_id_cat', 
             'glimepiride.bi']:
    

    devonDF[item] = le.fit_transform(devonDF[item])

In [162]:
for var in list(devonDF.columns):
    devonDF[var] = pd.to_numeric(devonDF[var])

In [172]:
devontrainX = devonDF[devonDF['IsTrain']==1].copy().drop('readmitted', axis=1)
devontrainY = devonDF[devonDF['IsTrain']==1].copy()['readmitted']
devontrainY[devontrainY<0] = 0
devontrainY[devontrainY>0] = 1
devontestX = devonDF[devonDF['IsTrain']==0].copy().drop('readmitted', axis=1)
devontestY = devonDF[devonDF['IsTrain']==0].copy()['readmitted']
devontestY[devontestY<0] = 0
devontestY[devontestY>0] = 1

from xgboost.sklearn import XGBClassifier as xgb

xgb2 = xgb()
xgb2.set_params(n_estimators=500, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, max_delta_step=5)

xgb2.fit(devontrainX, devontrainY)

predictDXB = xgb2.predict(devontestX)
predictprobsDXB = xgb2.predict_proba(devontestX)

C:\Users\Dave\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [174]:
AUC(devontestY, predictprobsDXB[:,1])

0.6785622482174802

In [176]:
from sklearn.linear_model import LogisticRegression as lgr

lgr = lgr()
lgr.set_params(C=0.1)

lgr.fit(devontrainX, devontrainY)

predicttestprobsLGRD = lgr.predict_proba(devontestX)

AUC(devontestY, predicttestprobsLGRD[:,1])

0.6576307436751204

In [177]:
from sklearn.ensemble import RandomForestClassifier as rfc

rfc = rfc()
rfc.set_params(n_estimators=1000, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', max_depth=60)

rfc.fit(devontrainX, devontrainY)

predicttestprobsRFD = rfc.predict_proba(devontestX)

AUC(devontestY, predicttestprobsRFD[:,1])

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x0000021893225710>>
Traceback (most recent call last):
  File "C:\Users\Dave\Anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


0.6602773262812346

In [179]:
###Feature names mismatch with Tim's data
timtrain = pd.read_csv('../Tim/multi_tim_12.1.csv')
timtest = pd.read_csv('../Tim/test_tim_12.5.csv')

timtrainX = timtrain.drop('readmitted_<30', axis=1)
timtrainY = timtrain['readmitted_<30']

timtestX = timtest.drop(['readmitted_>30', 'readmitted_<30'], axis=1)
timtestY = timtest['readmitted_<30']

from xgboost.sklearn import XGBClassifier as xgb

xgb2 = xgb()
xgb2.set_params(n_estimators=500, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, max_delta_step=5)

xgb2.fit(timtrainX, timtrainY)

predictTXB = xgb2.predict(timtestX)
predictprobsTXB = xgb2.predict_proba(timtestX)

ValueError: feature_names mismatch: ['Unnamed: 0', 'age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'race_Caucasian', 'race_AfricanAmerican', 'race_Other', 'race_Asian', 'race_Hispanic', 'gender_Male', 'gender_Unknown/Invalid', 'admission_type_id_Unknown', 'admission_type_id_Emergency', 'admission_type_id_Urgent', 'admission_type_id_Elective', 'discharge_disposition_id_Unknown', 'discharge_disposition_id_Discharged to home', 'discharge_disposition_id_Discharged/transferred to SNF', 'discharge_disposition_id_Discharged/transferred to home with home health service', 'discharge_disposition_id_Discharged/transferred to another short term hospital', 'discharge_disposition_id_Discharged/transferred to another type of inpatient care institution', 'discharge_disposition_id_Discharged/transferred to ICF', 'discharge_disposition_id_Discharged/transferred/referred to this institut

In [183]:
###Feature names mismatch with Anisha's data; column name issues
anishatrain = pd.read_csv('../anisha/dataset_diabetes/new_train.csv', index_col=0)
anishatest = pd.read_csv('../anisha/dataset_diabetes/new_test.csv', index_col=0)

anishatrainX = anishatrain.drop('readmitted_Yes', axis=1)
anishatrainY = anishatrain['readmitted_Yes']

anishatestX = anishatest.drop('readmitted_Yes', axis=1)
anishatestY = anishatest['readmitted_Yes']

from xgboost.sklearn import XGBClassifier as xgb

xgb2 = xgb()
xgb2.set_params(n_estimators=500, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, max_delta_step=5)

xgb2.fit(anishatrainX, anishatrainY)

predictAXB = xgb2.predict(anishatestX)
predictprobsAXB = xgb2.predict_proba(anishatestX)

ValueError: feature_names may not contain [, ] or <

In [184]:
anishacols = list(anishatrain.columns)

In [195]:
anishacols[9] = 'Age2'
anishacols[10] = 'Age3'
anishacols[11] = 'Age4'
anishacols[12] = 'Age5'
anishacols[13] = 'Age6'
anishacols[14] = 'Age7'
anishacols[15] = 'Age8'
anishacols[16] = 'Age9'

In [196]:
anishatrain.columns = anishacols
anishatest.columns = anishacols

In [198]:
anishatrainX = anishatrain.drop('readmitted_Yes', axis=1)
anishatrainX = anishatrainX.drop('encounter_id', axis=1)
anishatrainY = anishatrain['readmitted_Yes']

anishatestX = anishatest.drop('readmitted_Yes', axis=1)
anishatestX = anishatestX.drop('encounter_id', axis=1)
anishatestY = anishatest['readmitted_Yes']

from xgboost.sklearn import XGBClassifier as xgb

xgb2 = xgb()
xgb2.set_params(n_estimators=500, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, max_delta_step=5)

xgb2.fit(anishatrainX, anishatrainY)

predictAXB = xgb2.predict(anishatestX)
predictprobsAXB = xgb2.predict_proba(anishatestX)

C:\Users\Dave\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [200]:
AUC(anishatestY, predictprobsAXB[:,1])

0.675882142752384